In [ ]:
### CROWN DETERMINATION

In [ ]:
import pandas as pd
import math
import os
import numpy as np
from argparse import Namespace
import pathlib
import matplotlib as plt
import matplotlib.pyplot as mplt
#from shapely import geometry
from mpl_toolkits.mplot3d import Axes3D
import scipy.io as sio
import matplotlib.cm as cmx
import itertools

In [ ]:
dataDir= ""
qsm=[]
for file in os.listdir(dataDir): #or dataDir
    qsm.append(sio.loadmat(dataDir + file))

In [ ]:
#make dataframe
Bord=[]
X=[]
Y=[]
Z=[]
cCHI=[]
cPAR=[]
cExt=[]
TreeData=[]
ax1=[]
ax2=[]
ax3=[]
CLen=[]
rad=[]
BAng=[]
BOB=[]
FCB=[]
BLen=[]
BChi=[]
CiB=[]
BoC=[]
for tree in qsm:
    Bord.append(tree['BoC'][:,1])
    X.append(tree['Sta'][:,0])
    Y.append(tree['Sta'][:,1])
    Z.append(tree['Sta'][:,2])
    cCHI.append(tree['CChi'])
    cPAR.append(tree['CPar'])
    cExt.append(tree['CExt'])
    TreeData.append(tree['TreeData'])
    ax1.append(tree['Axe'][:,0])
    ax2.append(tree['Axe'][:,1])
    ax3.append(tree['Axe'][:,2])
    CLen.append(tree['Len'][:,0])
    rad.append(tree['Rad'][:,0])
    BAng.append(tree['BAng'][:,0])
    BOB.append(tree['BOrd'][:,0])
    FCB.append(tree['FCB'][:,0])
    BLen.append(tree['BLen'][:,0])
    BChi.append(tree['BChi'])
    CiB.append(tree['CiB'])
    BoC.append(tree['BoC'])
data=pd.DataFrame({'BoC':BoC, 'CiB':CiB, 'BChi':BChi, 'BLen':BLen,'FCB':FCB,'branch_order_b':BOB,'BAng':BAng, 
                    'branch_order':Bord,'cyl_rad':rad,'X':X,'Y':Y,'Z':Z,'cCHI':cCHI,'cEXT':cExt, 'cPAR':cPAR,
                    'TreeData':TreeData,'ax1':ax1,'ax2':ax2,'ax3':ax3,'cyl_length':CLen})

In [ ]:
def crown(row):
    tree = pd.DataFrame({'branch_order':row.branch_order,'X':row.X,'Y':row.Y, 'Z':row.Z,'parents':row.cPAR[:,0], 'children':row.cCHI[:,0], 'extension':row.cEXT[:,0], 'crown':[0]*len(row.cEXT),'Height':row.TreeData[3]}, index=range(1, len(row.branch_order)+1))
    order_zero=tree.loc[row.branch_order==0,:]
    if max(tree.loc[:,'branch_order']) >= 3:
        order_threshold=3
    else:
        order_threshold=max(tree.loc[:,'branch_order'])

    x=tree.loc[:,'branch_order']>=order_threshold
    ind=list(itertools.compress(range(len(x)), x))
    ind2=[1+x for x in ind]
    tree.loc[ind2,'crown']=1 #matlabindexen

    #parents adden
    #tree
    CR=ind2
    CR_old=[]
    while CR !=CR_old:
        CR_old=CR
        tree.loc[tree.loc[CR,'parents'],'crown']=1
        tree.loc[range(1,len(order_zero)+1),'crown']=0
        cr=tree.loc[:,'crown']==1
        CR=[1+cr for cr in list(itertools.compress(range(len(cr)), cr))]
        #delete CRs in list part from trunk
        for t in range(1,len(order_zero)+1):
            if t in CR:
                CR.remove(t)
            else:
                pass
    #lowest point with parent 0, 
    t1=tree.loc[tree.loc[:,'crown']==1,:]
    t2=t1.loc[t1.loc[:,'branch_order']==1,:]
    r=[]
    for t in t2.loc[:,'parents']:
        if tree.loc[t,'branch_order']==0:
            r.append(t2.loc[t2.loc[:,'parents']==t,'Z'].tolist()[0])
    height=min(r) 

    x=tree.loc[:,'Z']>=height
    ind=list(itertools.compress(range(len(x)), x))
    ind2=[1+x for x in ind]
    tree.loc[ind2,'crown']=1
    tree.loc[range(1,len(order_zero)+1),'crown']=0

    #append children
    trial=[]
    while tree.loc[:,'crown'].tolist() != trial:
        trial=tree.loc[:,'crown'].tolist()
        childs=tree.loc[tree.loc[:,'crown']==1,'children'].tolist()
        for i in childs:
            if i.size>0:
                for j in i:
                    tree.loc[j,'crown']==1
    # tree = create_tree()
    return tree

In [ ]:
DATA=[]
k=0
for i in range(len(data)):
    k=k+1
    DATA.append(crown(data.loc[i]))